<a href="https://colab.research.google.com/github/amamra-abdenour/Deep_Learning_Python/blob/master/Pickle_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
import cv2
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

In [0]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

# Could not get the download to work through python for some reason.
# Completed it manually through the web browser.
train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)
train_filename = 'notMNIST_large.tar.gz'
test_filename = 'notMNIST_small.tar.gz'

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


In [0]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

Extracting data for notMNIST_large. This may take a while. Please wait.
['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
Extracting data for notMNIST_small. This may take a while. Please wait.
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


In [0]:
from IPython.display import Image
Image(os.getcwd() + '/notMNIST_small/A/MDEtMDEtMDAudHRm.png')

In [0]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letter(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  once = True
  image_index = 0
  print(folder)
  for image in os.listdir(folder):
    image_file = os.path.join(folder, image)
    try:
      read_image = np.float32(cv2.imread(image_file,cv2.IMREAD_GRAYSCALE))
      image_data = (read_image - pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        print(image_index)
        #raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      else:
        dataset[image_index, :, :] = image_data
        image_index += 1
    except IOError as e:
      print('Could not read file - it\'s ok, skipping.')
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

NameError: ignored

In [0]:
image_w = 28
image_h = 28
def pickle_image_folder(folder):
  #The folder containing all the images
  image_files = os.listdir(folder) #List of images in the precedent folder
  #print(image_files)
  dataset = np.ndarray(shape=(len(image_files),image_w,image_h),dtype = np.float32)
  image_index = 0
  for img in os.listdir(folder): #For all the images
    image_file = os.path.join(folder,img) #build a single image path
    #print(image_file)
    try:
      image = np.float32(cv2.imread (image_file,0))#read image file
      if (image.shape != (image_w, image_h)): #if it has not the right shape discard it
        print('Error' , image_file, 'has shape :', image.shape)
      else:
        dataset[image_index, :, :] = image #otherwise insert a numpy array into the dataset
        image_index += 1
    except IOError as err:
      print('Could not read file, skipping reading it')
  print('Number of images {0}, read images {1}'.format(len(image_files),image_index ))
  return dataset
    
image_data = pickle_image_folder('notMNIST_small/D')
print(image_data.shape)


FileNotFoundError: ignored

In [0]:
from google.colab.patches import cv2_imshow
idx = np.random.randint(0,image_data.shape[0] - 1,10)
for i in idx:
  cv2_imshow(image_data[i])
cv2.waitKey(0)

-1

In [0]:
for i in range(10):
  cv2_imshow(image_data[i])
cv2.waitKey(0)

-1

In [0]:
def save_dataset_file(dataset,folder):
  filename = folder + '.pickle'
  try:
    with open(filename,'wb') as f:
      pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      print('File ',filename,' saved with success')
  except Exception as e:
      print('Unable to save file ',filename,e)
save_dataset_file(image_data,'D')

File  D.pickle  saved with success


In [0]:
loaded_dataset = pickle.load(open('D.pickle','rb'))
print(type(loaded_dataset), loaded_dataset.shape)

<class 'numpy.ndarray'> (1873, 28, 28)


In [0]:
from google.colab.patches import cv2_imshow
idx = np.random.randint(0,image_data.shape[0] - 1,10)
for i in idx:
  cv2_imshow(loaded_dataset[i])
cv2.waitKey(0)

-1

In [0]:
for i in range(10):
  cv2_imshow(loaded_dataset[i])
cv2.waitKey(0)

NameError: ignored

In [0]:
import matplotlib.pyplot as plt
plt.imshow(loaded_dataset[0,:,:])

NameError: ignored